In [1]:
import polars as pl
import numpy as np

In [2]:
# Load the data
data0  =  pl.read_parquet("data/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=6/part-0.parquet")
data1  =  pl.read_parquet('data/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=7/part-0.parquet')
data2  =  pl.read_parquet('data/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=8/part-0.parquet')
data3  =  pl.read_parquet('data/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=9/part-0.parquet')

In [3]:
# remove the nan values
def replace_nan_with_mean_or_zero(column):
    mean_value = column.mean()
    if pd.notnull(mean_value):
        return column.fillna(mean_value)
    else:
        return column.fillna(0)

In [5]:
train_data = pl.concat([data0, data1,data2,data3])  # 3 partitions
del data0 ,data1, data2, data3 # free up memory

In [8]:
train_data = train_data.with_columns([
    pl.col(column).fill_null(pl.col(column).mean()) for column in train_data.columns
])

In [9]:
# define the columns
lag_cols_original = ["date_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]

In [14]:
#new lag data
lags = train_data.select(lag_cols_original)
lags = lags.rename({ f"responder_{idx}" : f"responder_{idx}_lag_1" for idx in range(9)})

lags = lags.with_columns((pl.col("date_id") + 1).alias("date_id"))
lags = lags.groupby(["date_id", "symbol_id"]).agg([pl.all().last()]) # pick up last record of previous date
train_data = train_data.join(lags, on=["date_id", "symbol_id"], how="left")
del lags ,lag_cols_original
train_data = train_data.fill_null(0)

AttributeError: 'DataFrame' object has no attribute 'groupby'

In [8]:
train_data.reset_index(drop=True)

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,responder_8,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
0,1190,0,0,2.888326,0.928983,0.441290,1.352455,1.301461,2.302733,-0.231061,...,0.546211,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1190,0,1,5.419313,1.793786,0.198769,1.165048,0.392095,2.187486,-0.220963,...,-0.907372,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1190,0,2,1.286522,1.177853,0.071345,1.043871,1.246540,1.825975,-0.283726,...,1.000657,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1190,0,3,1.223723,1.093739,-0.049899,0.715656,0.709871,1.136459,-0.194208,...,2.265868,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1190,0,4,1.949932,1.186918,0.294662,0.873467,0.810031,2.020650,-0.113065,...,-2.002821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18750155,1698,967,34,3.242493,2.525160,-0.721981,2.544025,2.477615,0.417557,0.785812,...,-0.252461,0.501321,0.905332,-0.819582,-0.564046,-0.223018,-0.283954,-0.045938,0.009797,-0.102538
18750156,1698,967,35,1.079139,1.857906,-0.790646,2.745439,2.339877,0.845065,0.651370,...,-0.460630,-1.113053,0.697190,-1.619031,-1.222743,-0.706082,-0.291133,0.167733,0.099704,0.324610
18750157,1698,967,36,1.033172,2.515527,-0.672298,2.289250,2.521592,0.255077,0.919892,...,-0.125170,-1.019353,-0.460962,-2.026678,-0.848606,-0.305448,-1.256913,-0.109359,-0.027474,-0.253956
18750158,1698,967,37,1.243116,2.663298,-0.889112,2.313155,3.101428,0.324454,0.618944,...,-0.256395,0.235850,0.556479,0.618944,-0.243765,-0.108361,-0.260777,-0.486923,-0.275566,-1.020708


In [10]:
train_data.to_parquet('data/df.parquet')

In [11]:
num_val = 1670
val_set = train_data[train_data.date_id >= num_val]
val_set.to_parquet('data/validate_data.parquet')

In [15]:
train_data.drop(train_data.loc[train_data.date_id >= num_val].index, inplace=True)

train_data.to_parquet('data/train_data.parquet')